In [24]:
import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [25]:
if 0:
    print('Load street graph')
    G = snman.io.load_street_graph(
        os.path.join(outputs_path, 'G_edges.gzip'),
        os.path.join(outputs_path, 'G_nodes.gzip'),
        crs=CRS_internal
    )
    
    snman.street_graph.organize_edge_directions(G)
    
    nodes, edges = oxc.graph_to_gdfs(G)

In [26]:
# Import Statent grid
statent = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
)
statent.rename(columns={'RELI': 'id'}, inplace=True)
statent

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,VOLLZEITAEQ_SEKTOR1,VOLLZEITAEQ_SEKTOR1_FRAUEN,VOLLZEITAEQ_SEKTOR1_MAENNER,VOLLZEITAEQ_SEKTOR2,VOLLZEITAEQ_SEKTOR2_FRAUEN,VOLLZEITAEQ_SEKTOR2_MAENNER,VOLLZEITAEQ_SEKTOR3,VOLLZEITAEQ_SEKTOR3_FRAUEN,VOLLZEITAEQ_SEKTOR3_MAENNER,geometry
0,2021,2023,2632200,1247300,63222473,4,0,4,4,7,...,0.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,0.0,POINT (2632200.000 1247300.000)
1,2021,2023,2632300,1248100,63232481,4,4,0,0,4,...,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (2632300.000 1248100.000)
2,2021,2023,2632400,1247300,63242473,4,0,0,4,5,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2632400.000 1247300.000)
3,2021,2023,2632400,1247500,63242475,4,0,4,0,4,...,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,POINT (2632400.000 1247500.000)
4,2021,2023,2632500,1247200,63252472,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2632500.000 1247200.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58739,2021,2023,2728300,1222500,72832225,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2728300.000 1222500.000)
58740,2021,2023,2728300,1222900,72832229,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,POINT (2728300.000 1222900.000)
58741,2021,2023,2728400,1222700,72842227,4,0,0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,POINT (2728400.000 1222700.000)
58742,2021,2023,2728500,1222500,72852225,4,4,0,0,4,...,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (2728500.000 1222500.000)


In [27]:
# read statpop that has been pre-joined with statent
statpop = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_with_statent_reduced_columns.gzip')
)

statpop.head()

,statyear,yearofbirth,sex,maritalstatus,residencepermit,age,residentpermit,federalbuildingid,geometry,index_right,statent_id,distance_to_statent_point
record,,,,,,,,,,,,
12,2017,1962,1,2,302,55,3,896883164,POINT (2674380.001 1250094.001),18914.0,67442501.0,20.879700
15,2017,1966,2,1,70102,51,7,899882330,POINT (2672299.001 1251208.001),17671.0,67232512.0,8.063317
16,2017,1984,1,1,13,33,13,899882659,POINT (2673496.001 1250355.001),NaN,NaN,NaN
20,2017,1992,1,2,201,25,2,899882087,POINT (2671984.001 1250827.001),17414.0,67202508.0,31.385383
21,2017,1994,2,2,201,23,2,899882087,POINT (2671984.001 1250827.001),17414.0,67202508.0,31.385383


In [28]:
# filter statpop for one single statent cell
statent_id = 68252512
statpop_filtered = statpop.query(f'statent_id == {statent_id}')
statpop_filtered.head()

,statyear,yearofbirth,sex,maritalstatus,residencepermit,age,residentpermit,federalbuildingid,geometry,index_right,statent_id,distance_to_statent_point
record,,,,,,,,,,,,
234507,2017,1981,2,2,-2,36,-2,597939907,POINT (2682548.001 1251183.001),28059.0,68252512.0,50.921708
274900,2017,1972,2,2,-2,45,-2,899849080,POINT (2682502.001 1251169.001),28059.0,68252512.0,31.063351
379693,2017,2007,1,1,-2,10,-2,597939911,POINT (2682545.001 1251248.001),28059.0,68252512.0,65.796390
417369,2017,2010,1,1,-2,7,-2,597939906,POINT (2682532.001 1251207.001),28059.0,68252512.0,32.757524
637853,2017,1929,2,3,-2,88,-2,597939911,POINT (2682545.001 1251248.001),28059.0,68252512.0,65.796390


In [31]:
# filter statent for cells within distance limit
origin = statent.set_index('id').geometry[statent_id]
origins = geopandas.GeoDataFrame(
    {
        "id": [statent_id],
        "geometry": [origin]
    },
    crs="EPSG:2056",
)
statent_filtered = statent[statent['geometry'].within(
    origin.buffer(1000)
)]
statent_filtered

,ERHJAHR,PUBJAHR,E_KOORD,N_KOORD,id,ARBEITSSTAETEN_TOTAL,ARBEITSSTAETEN_SEKTOR1,ARBEITSSTAETEN_SEKTOR2,ARBEITSSTAETEN_SEKTOR3,BESCHAEFTIGTE_TOTAL,...,VOLLZEITAEQ_SEKTOR1,VOLLZEITAEQ_SEKTOR1_FRAUEN,VOLLZEITAEQ_SEKTOR1_MAENNER,VOLLZEITAEQ_SEKTOR2,VOLLZEITAEQ_SEKTOR2_FRAUEN,VOLLZEITAEQ_SEKTOR2_MAENNER,VOLLZEITAEQ_SEKTOR3,VOLLZEITAEQ_SEKTOR3_FRAUEN,VOLLZEITAEQ_SEKTOR3_MAENNER,geometry
26621,2021,2023,2681700,1251600,68172516,4,0,0,4,4,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.000000,4.000000,0.000000,POINT (2681700.000 1251600.000)
26622,2021,2023,2681700,1251700,68172517,4,0,0,4,4,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.000000,4.000000,4.000000,POINT (2681700.000 1251700.000)
26774,2021,2023,2681800,1251500,68182515,4,0,4,4,8,...,0.0,0.0,0.0,4.000000,0.000000,4.000000,4.000000,4.000000,4.000000,POINT (2681800.000 1251500.000)
26775,2021,2023,2681800,1251600,68182516,4,0,0,4,4,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.000000,4.000000,4.000000,POINT (2681800.000 1251600.000)
26776,2021,2023,2681800,1251700,68182517,4,0,0,4,4,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.000000,0.000000,4.000000,POINT (2681800.000 1251700.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29630,2021,2023,2683400,1251200,68342512,5,0,0,5,115,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,56.664819,34.093113,22.571706,POINT (2683400.000 1251200.000)
29631,2021,2023,2683400,1251300,68342513,12,0,4,11,22,...,0.0,0.0,0.0,4.000000,4.000000,0.000000,15.416539,5.500922,9.915617,POINT (2683400.000 1251300.000)
29632,2021,2023,2683400,1251400,68342514,48,0,4,44,410,...,0.0,0.0,0.0,29.566898,4.000000,27.212349,308.259268,125.409646,182.849621,POINT (2683400.000 1251400.000)
29633,2021,2023,2683400,1251500,68342515,40,0,4,39,327,...,0.0,0.0,0.0,7.044031,5.494031,4.000000,230.480890,122.484848,107.996042,POINT (2683400.000 1251500.000)


In [7]:
transport_network = r5py.TransportNetwork(
    os.path.join(outputs_path, 'before_oneway_links_default.osm.pbf'),
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'vbz_2024.zip')
)

In [14]:
tt_computers = {}
common_settings = {
    'origins': origins,
    'destinations': statent_filtered,
    'departure': datetime.datetime(2024, 2, 22, 8, 30),
}

tt_computers[MODE_FOOT] = r5py.TravelTimeMatrixComputer(
    transport_network,
    transport_modes=[r5py.TransportMode.WALK],
    **common_settings
)

tt_computers[MODE_CYCLING] = r5py.TravelTimeMatrixComputer(
    transport_network,
    transport_modes=[r5py.TransportMode.BICYCLE],
    **common_settings
)

tt_computers[MODE_PRIVATE_CARS] = r5py.TravelTimeMatrixComputer(
    transport_network,
    transport_modes=[r5py.TransportMode.CAR],
    **common_settings
)

tt_computers[MODE_TRANSIT] = r5py.TravelTimeMatrixComputer(
    transport_network,
    transport_modes=[r5py.TransportMode.TRANSIT],
    **common_settings
)

# travel time matrix calculation: compute travel times, add a mode label
def tt_calculation(tt_computer, mode):
    tt_matrix = tt_computer.compute_travel_times()
    tt_matrix['mode'] = mode
    return tt_matrix

# calculate a travel time matrix for each mode
tt_matrices = {mode: tt_calculation(tt_computer, mode) for mode, tt_computer in tt_computers.items()}

# merge all travel time matrices into one
tt_matrix = pd.concat(tt_matrices.values())

# add base travel time to avoid 0 values
tt_matrix['travel_time'] = tt_matrix['travel_time'] + 0.1
tt_matrix

,from_id,to_id,travel_time,mode
0,68252512,68172516,20.1,foot
1,68252512,68172517,19.1,foot
2,68252512,68182515,17.1,foot
3,68252512,68182516,17.1,foot
4,68252512,68182517,17.1,foot
...,...,...,...,...
192,68252512,68342512,21.1,transit
193,68252512,68342513,21.1,transit
194,68252512,68342514,19.1,transit
195,68252512,68342515,19.1,transit


In [15]:
statpop_reduced = statpop_filtered.reset_index()[['record', 'age', 'statent_id', 'geometry']]
statpop_reduced.set_index('record', inplace=True)
statpop_reduced.head()

,age,statent_id,geometry
record,,,
234507,36,68252512.0,POINT (2682548.001 1251183.001)
274900,45,68252512.0,POINT (2682502.001 1251169.001)
379693,10,68252512.0,POINT (2682545.001 1251248.001)
417369,7,68252512.0,POINT (2682532.001 1251207.001)
637853,88,68252512.0,POINT (2682545.001 1251248.001)


In [16]:
accessibility_costs = pd.merge(statpop_reduced.reset_index(), tt_matrix, how='left', left_on='statent_id', right_on='from_id')
accessibility_costs = accessibility_costs[accessibility_costs['travel_time'].notna()]
accessibility_costs

,record,age,statent_id,geometry,from_id,to_id,travel_time,mode
0,234507,36,68252512.0,POINT (2682548.001 1251183.001),68252512,68172516,20.1,foot
1,234507,36,68252512.0,POINT (2682548.001 1251183.001),68252512,68172517,19.1,foot
2,234507,36,68252512.0,POINT (2682548.001 1251183.001),68252512,68182515,17.1,foot
3,234507,36,68252512.0,POINT (2682548.001 1251183.001),68252512,68182516,17.1,foot
4,234507,36,68252512.0,POINT (2682548.001 1251183.001),68252512,68182517,17.1,foot
...,...,...,...,...,...,...,...,...
113467,12723489,48,68252512.0,POINT (2682455.001 1251225.001),68252512,68342512,21.1,transit
113468,12723489,48,68252512.0,POINT (2682455.001 1251225.001),68252512,68342513,21.1,transit
113469,12723489,48,68252512.0,POINT (2682455.001 1251225.001),68252512,68342514,19.1,transit
113470,12723489,48,68252512.0,POINT (2682455.001 1251225.001),68252512,68342515,19.1,transit


In [17]:
accessibility_costs['record_to_id'] = accessibility_costs['record'].astype(str) + accessibility_costs['to_id'].astype(str)
accessibility_costs

,record,age,statent_id,geometry,from_id,to_id,travel_time,mode,record_to_id
0,234507,36,68252512.0,POINT (2682548.001 1251183.001),68252512,68172516,20.1,foot,23450768172516
1,234507,36,68252512.0,POINT (2682548.001 1251183.001),68252512,68172517,19.1,foot,23450768172517
2,234507,36,68252512.0,POINT (2682548.001 1251183.001),68252512,68182515,17.1,foot,23450768182515
3,234507,36,68252512.0,POINT (2682548.001 1251183.001),68252512,68182516,17.1,foot,23450768182516
4,234507,36,68252512.0,POINT (2682548.001 1251183.001),68252512,68182517,17.1,foot,23450768182517
...,...,...,...,...,...,...,...,...,...
113467,12723489,48,68252512.0,POINT (2682455.001 1251225.001),68252512,68342512,21.1,transit,1272348968342512
113468,12723489,48,68252512.0,POINT (2682455.001 1251225.001),68252512,68342513,21.1,transit,1272348968342513
113469,12723489,48,68252512.0,POINT (2682455.001 1251225.001),68252512,68342514,19.1,transit,1272348968342514
113470,12723489,48,68252512.0,POINT (2682455.001 1251225.001),68252512,68342515,19.1,transit,1272348968342515


In [18]:
# group accessibility by person, providing a list of travel times to each statent destination

ag = accessibility_costs.reset_index().groupby(['record_to_id']).agg({
    'travel_time': list,
    'mode': list,
    #'sex': 'first',
    'age': 'first',
    #'maritalstatus': 'first',
    #'residencepermit': 'first',
    #'residentpermit': 'first'
    'record': 'first',
    'to_id': 'first'
})

# add destination information from statent
ag = pd.merge(ag.reset_index(), statent.reset_index()[['id', 'VOLLZEITAEQ_TOTAL']], left_on='to_id', right_on='id')

ag

,record_to_id,travel_time,mode,age,record,to_id,id,VOLLZEITAEQ_TOTAL
0,1012639168172516,"[20.1, 22.1, 16.1]","[foot, cycling, transit]",45,10126391,68172516,68172516,4.000000
1,1012639168172517,"[19.1, 23.1, 15.1]","[foot, cycling, transit]",45,10126391,68172517,68172517,4.000000
2,1012639168182515,"[17.1, 21.1, 15.1]","[foot, cycling, transit]",45,10126391,68182515,68182515,5.309428
3,1012639168182516,"[17.1, 22.1, 13.1]","[foot, cycling, transit]",45,10126391,68182516,68182516,4.000000
4,1012639168182517,"[17.1, 10.1, 13.1]","[foot, cycling, transit]",45,10126391,68182517,68182517,4.000000
...,...,...,...,...,...,...,...,...
28363,999058068342512,"[22.1, 9.1, 21.1]","[foot, cycling, transit]",37,9990580,68342512,68342512,56.664819
28364,999058068342513,"[23.1, 12.1, 21.1]","[foot, cycling, transit]",37,9990580,68342513,68342513,16.359032
28365,999058068342514,"[23.1, 10.1, 19.1]","[foot, cycling, transit]",37,9990580,68342514,68342514,337.826166
28366,999058068342515,"[23.1, 9.1, 19.1]","[foot, cycling, transit]",37,9990580,68342515,68342515,237.524921


In [20]:
# create a dictionary with travel options for each person and statent destination
ag['travel_options'] = ag.apply(lambda row: dict(zip(row['mode'], row['travel_time'])), axis=1)
ag

,record_to_id,travel_time,mode,age,record,to_id,id,VOLLZEITAEQ_TOTAL,travel_options
0,1012639168172516,"[20.1, 22.1, 16.1]","[foot, cycling, transit]",45,10126391,68172516,68172516,4.000000,"{'foot': 20.1, 'cycling': 22.1, 'transit': 16.1}"
1,1012639168172517,"[19.1, 23.1, 15.1]","[foot, cycling, transit]",45,10126391,68172517,68172517,4.000000,"{'foot': 19.1, 'cycling': 23.1, 'transit': 15.1}"
2,1012639168182515,"[17.1, 21.1, 15.1]","[foot, cycling, transit]",45,10126391,68182515,68182515,5.309428,"{'foot': 17.1, 'cycling': 21.1, 'transit': 15.1}"
3,1012639168182516,"[17.1, 22.1, 13.1]","[foot, cycling, transit]",45,10126391,68182516,68182516,4.000000,"{'foot': 17.1, 'cycling': 22.1, 'transit': 13.1}"
4,1012639168182517,"[17.1, 10.1, 13.1]","[foot, cycling, transit]",45,10126391,68182517,68182517,4.000000,"{'foot': 17.1, 'cycling': 10.1, 'transit': 13.1}"
...,...,...,...,...,...,...,...,...,...
28363,999058068342512,"[22.1, 9.1, 21.1]","[foot, cycling, transit]",37,9990580,68342512,68342512,56.664819,"{'foot': 22.1, 'cycling': 9.1, 'transit': 21.1}"
28364,999058068342513,"[23.1, 12.1, 21.1]","[foot, cycling, transit]",37,9990580,68342513,68342513,16.359032,"{'foot': 23.1, 'cycling': 12.1, 'transit': 21.1}"
28365,999058068342514,"[23.1, 10.1, 19.1]","[foot, cycling, transit]",37,9990580,68342514,68342514,337.826166,"{'foot': 23.1, 'cycling': 10.1, 'transit': 19.1}"
28366,999058068342515,"[23.1, 9.1, 19.1]","[foot, cycling, transit]",37,9990580,68342515,68342515,237.524921,"{'foot': 23.1, 'cycling': 9.1, 'transit': 19.1}"


In [21]:
# run the mode choice model to calculate each person's generalized cost to each statent destination

def calculate_behavioral_cost(private_cars=9999999, transit=9999999, cycling=9999999, foot=9999999, age=None):
    """
    Calculates individual generalized cost based on the personal properties and mode choice.
    Based on mode choice model in
    
    Hörl, S., M. Balać and K.W. Axhausen (2019)
    Pairing discrete mode choice models and agent-based transport simulation with MATSim,
    paper presented at the 98th Annual Meeting of the Transportation Research Board (TRB 2019),
    Washington DC, January 2019. 
    
    Returns a tuple of mode specific choice probabilities (dict) and the resulting behavioral travel time (float)

    Parameters
    ----------
    private_cars: float
        travel time in minutes
    transit: float
        travel time in minutes
    cycling: float
        travel time in minutes
    foot: float
        travel time in minutes
    age: int
        age in years

    Returns
    -------
    (dict, float)
    """
    
    # mode-specific costs (travel times in minutes)
    tt = {
        MODE_PRIVATE_CARS: private_cars,
        MODE_TRANSIT: transit,
        MODE_CYCLING: cycling,
        MODE_FOOT: foot
    }
    
    # mode choice model parameters from Hörl et al. (2019), pp. 11, Table 1
    # t: time
    
    alpha_car =                    0.827   # [-]
    beta_tt_car =                 -0.0667  # [min-1]
    
    alpha_pt =                     0.0     # [-]
    beta_transfer =               -0.17    # [-]
    beta_in_vehicle_t =           -0.0192  # [min-1]
    beta_transfer_t =             -0.0384  # [min-1]
    beta_access_egress_t =        -0.0804  # [min-1]
    
    alpha_bike =                  -0.1     # [-]
    beta_tt_bike =                -0.0805  # [min-1]
    beta_age_bike =               -0.0496  # [years-1]
    
    alpha_walk =                   0.63    # [-]
    beta_tt_walk =                -0.141   # [min-1]
    
    beta_cost =                   -0.126   # [CHF-1]
    lambda_factor =               -0.4     # [-1]
    phi_avg_distance =            40.0     # [km]
    
    phi_parking_search_penalty =   6.0     # [min]
    phi_access_egress_walk_t =     5.0     # [min]
    
    # surrogate parameters to replace missing information (R5 provides only travel time)
    avg_speed_car =               25.0     # [kmh]
    detour_parameter_car =         1.3     # [-]
    x_crowfly_distance = tt[MODE_PRIVATE_CARS] / 60 * avg_speed_car / detour_parameter_car
    x_cost_car = tt[MODE_PRIVATE_CARS] * 0.2
    x_cost_pt = tt[MODE_TRANSIT] * 0.2
    
    x_transfers = round(tt[MODE_TRANSIT] / 30) if tt[MODE_TRANSIT] is not np.inf else np.inf
    x_transfer_t = x_transfers * 5
    x_access_egress_t = 5
    
    # mode-specific utilities from Hörl et al. (2019), pp. 10, eq. 3-6
    u = {
        MODE_PRIVATE_CARS: 
            alpha_car
            + beta_tt_car * tt[MODE_PRIVATE_CARS]
            + beta_tt_car * phi_parking_search_penalty
            + beta_tt_walk * phi_access_egress_walk_t
            + beta_cost * (x_crowfly_distance / phi_avg_distance)**lambda_factor * x_cost_car
        ,
        MODE_TRANSIT:
            alpha_pt
            + beta_transfer * x_transfers
            + beta_transfer_t * x_transfer_t
            + beta_access_egress_t * x_access_egress_t
            + beta_cost * (x_crowfly_distance / phi_avg_distance)**lambda_factor * x_cost_pt
        ,
        MODE_CYCLING:      
            alpha_bike
            + beta_tt_bike * tt[MODE_CYCLING]
            + beta_age_bike * max(0, age - 18)
        ,
        MODE_FOOT:
            alpha_walk
            + beta_tt_walk * tt[MODE_FOOT]
    }

    # mode-specific choice probabilities
    denominator = sum([np.exp(u_mode) for u_mode in u.values()])
    P = {mode: np.exp(u_mode) / denominator for mode, u_mode in u.items()}
    
    # total cost, after weighting the mode-specific cost by mode-specific choice probabilities
    tt_total = sum([tt[mode] * P[mode] for mode in tt.keys()])
    return P, tt_total

ag[['choice_probabilities', 'behavioral_cost']] = ag.apply(
    lambda row: calculate_behavioral_cost(**row['travel_options'], age=row['age']),
    axis=1,
    result_type='expand'
)

ag['accessibility_contribution'] = ag['behavioral_cost'] ** -0.7

ag

,record_to_id,travel_time,mode,age,record,to_id,id,VOLLZEITAEQ_TOTAL,travel_options,choice_probabilities,behavioral_cost,accessibility_contribution
0,1012639168172516,"[20.1, 22.1, 16.1]","[foot, cycling, transit]",45,10126391,68172516,68172516,4.000000,"{'foot': 20.1, 'cycling': 22.1, 'transit': 16.1}","{'private_cars': 0.0, 'transit': 0.75513528515...",17.209809,0.136445
1,1012639168172517,"[19.1, 23.1, 15.1]","[foot, cycling, transit]",45,10126391,68172517,68172517,4.000000,"{'foot': 19.1, 'cycling': 23.1, 'transit': 15.1}","{'private_cars': 0.0, 'transit': 0.73881131834...",16.380027,0.141247
2,1012639168182515,"[17.1, 21.1, 15.1]","[foot, cycling, transit]",45,10126391,68182515,68182515,5.309428,"{'foot': 17.1, 'cycling': 21.1, 'transit': 15.1}","{'private_cars': 0.0, 'transit': 0.68649831994...",15.983806,0.143689
3,1012639168182516,"[17.1, 22.1, 13.1]","[foot, cycling, transit]",45,10126391,68182516,68182516,4.000000,"{'foot': 17.1, 'cycling': 22.1, 'transit': 13.1}","{'private_cars': 0.0, 'transit': 0.76175323117...",14.281689,0.155473
4,1012639168182517,"[17.1, 10.1, 13.1]","[foot, cycling, transit]",45,10126391,68182517,68182517,4.000000,"{'foot': 17.1, 'cycling': 10.1, 'transit': 13.1}","{'private_cars': 0.0, 'transit': 0.70897804413...",13.481120,0.161879
...,...,...,...,...,...,...,...,...,...,...,...,...
28363,999058068342512,"[22.1, 9.1, 21.1]","[foot, cycling, transit]",37,9990580,68342512,68342512,56.664819,"{'foot': 22.1, 'cycling': 9.1, 'transit': 21.1}","{'private_cars': 0.0, 'transit': 0.64694558454...",18.374943,0.130329
28364,999058068342513,"[23.1, 12.1, 21.1]","[foot, cycling, transit]",37,9990580,68342513,68342513,16.359032,"{'foot': 23.1, 'cycling': 12.1, 'transit': 21.1}","{'private_cars': 0.0, 'transit': 0.69272988785...",19.524018,0.124911
28365,999058068342514,"[23.1, 10.1, 19.1]","[foot, cycling, transit]",37,9990580,68342514,68342514,337.826166,"{'foot': 23.1, 'cycling': 10.1, 'transit': 19.1}","{'private_cars': 0.0, 'transit': 0.66956419490...",17.484033,0.134943
28366,999058068342515,"[23.1, 9.1, 19.1]","[foot, cycling, transit]",37,9990580,68342515,68342515,237.524921,"{'foot': 23.1, 'cycling': 9.1, 'transit': 19.1}","{'private_cars': 0.0, 'transit': 0.65711614896...",17.106387,0.137022


In [22]:
# for each person, sum the accessibility contributions across all destinations
accessibility = ag.groupby('record').agg({
    'accessibility_contribution': 'sum'
})

accessibility.rename(columns={'accessibility_contribution': 'accessibility'}, inplace=True)

accessibility = accessibility.join(statpop_reduced)
accessibility = gpd.GeoDataFrame(accessibility, crs=2056)
accessibility

,accessibility,age,statent_id,geometry
record,,,,
234507,40.035531,36,68252512.0,POINT (2682548.001 1251183.001)
274900,39.905723,45,68252512.0,POINT (2682502.001 1251169.001)
379693,41.087569,10,68252512.0,POINT (2682545.001 1251248.001)
417369,41.087569,7,68252512.0,POINT (2682532.001 1251207.001)
637853,40.277562,88,68252512.0,POINT (2682545.001 1251248.001)
...,...,...,...,...
12328917,40.127557,33,68252512.0,POINT (2682545.001 1251248.001)
12387021,41.087569,10,68252512.0,POINT (2682548.001 1251183.001)
12421141,39.970165,39,68252512.0,POINT (2682450.001 1251222.001)


In [23]:
snman.io.export_gdf(
    accessibility,
    os.path.join(outputs_path, 'accessibility.gpkg')
)

In [ ]:
if 0:
    for column in ['transport_mode', 'travel_time']:
        travel_times[column] = travel_times[column].astype(str)
    travel_times.to_file(os.path.join(outputs_path, 'routes.gpkg'))

In [ ]:
tt_computers = {}
tt_computers[MODE_TRANSIT] = r5py.DetailedItinerariesComputer(
    transport_network,
    origins=origins,
    destinations=destinations,
    snap_to_network=False,
    departure=datetime.datetime(2024, 2, 22, 15, 30),
    transport_modes=[r5py.TransportMode.TRANSIT],
    max_time=datetime.timedelta(seconds=30),
    speed_walking=3.6,
    speed_cycling=12.0,
    max_public_transport_rides=1,
    max_bicycle_traffic_stress=3,
    breakdown=False,
)

r5py.RegionalTask

In [ ]:
tt_computers[MODE_TRANSIT].compute_travel_details()

In [ ]:
tt_computers = {}
tt_computers[MODE_CYCLING] = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=destinations,
    snap_to_network=False,
    departure=datetime.datetime(2024, 2, 22, 15, 30),
    transport_modes=[r5py.TransportMode.CAR],
)

In [ ]:
tt_computers[MODE_CYCLING].compute_travel_times()

In [ ]:
origins = geopandas.GeoDataFrame(
    {
        "id": ["origin"],
        "geometry": [G.nodes[335]['geometry']]
    },
    crs="EPSG:2056",
)

destinations = geopandas.GeoDataFrame(
    {
        "id": ["destination_1"],
        "geometry": [G.nodes[173]['geometry']]
    },
    crs="EPSG:2056",
)

In [38]:
tt_computers = {}
tt_computers[MODE_CYCLING] = r5py.DetailedItinerariesComputer(
    transport_network,
    origins=origins,
    destinations=statent_filtered,
    snap_to_network=False,
    departure=datetime.datetime(2024, 2, 22, 15, 30),
    transport_modes=[r5py.TransportMode.WALK],
    breakdown=True,
)

routes = tt_computers[MODE_CYCLING].compute_travel_details()
routes['transport_mode'] = routes['transport_mode'].astype(str)
routes['travel_time'] = routes['travel_time'].astype(str)
routes['wait_time'] = routes['wait_time'].astype(str)
snman.io.export_gdf(routes, os.path.join(outputs_path, 'routes.gpkg'))
routes

,from_id,to_id,option,segment,transport_mode,departure_time,distance,travel_time,wait_time,route,geometry
0,68252512,68172516,0,0,TransportMode.WALK,None,1175.195,0 days 00:19:38,None,None,"LINESTRING (8.532 47.406, 8.532 47.406, 8.531 ..."
1,68252512,68172517,0,0,TransportMode.WALK,None,1171.873,0 days 00:19:34,None,None,"LINESTRING (8.532 47.406, 8.532 47.406, 8.531 ..."
2,68252512,68182515,0,0,TransportMode.WALK,None,1061.506,0 days 00:17:43,None,None,"LINESTRING (8.532 47.406, 8.532 47.406, 8.531 ..."
3,68252512,68182516,0,0,TransportMode.WALK,None,1005.290,0 days 00:16:48,None,None,"LINESTRING (8.532 47.406, 8.532 47.406, 8.531 ..."
4,68252512,68182517,0,0,TransportMode.WALK,None,1045.558,0 days 00:17:28,None,None,"LINESTRING (8.532 47.406, 8.532 47.406, 8.531 ..."
...,...,...,...,...,...,...,...,...,...,...,...
192,68252512,68342512,0,0,TransportMode.WALK,None,1342.601,0 days 00:22:26,None,None,"LINESTRING (8.532 47.406, 8.532 47.406, 8.533 ..."
193,68252512,68342513,0,0,TransportMode.WALK,None,1404.102,0 days 00:23:28,None,None,"LINESTRING (8.532 47.406, 8.532 47.406, 8.533 ..."
194,68252512,68342514,0,0,TransportMode.WALK,None,1361.377,0 days 00:22:44,None,None,"LINESTRING (8.532 47.406, 8.532 47.406, 8.533 ..."
195,68252512,68342515,0,0,TransportMode.WALK,None,1372.342,0 days 00:22:56,None,None,"LINESTRING (8.532 47.406, 8.532 47.406, 8.533 ..."


In [35]:
tt_computers = {}
tt_computers[MODE_CYCLING] = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=statent_filtered,
    snap_to_network=False,
    departure=datetime.datetime(2024, 2, 22, 15, 30),
    transport_modes=[r5py.TransportMode.CAR],
    breakdown=True,
)

routes = tt_computers[MODE_CYCLING].compute_travel_times()
#routes['transport_mode'] = routes['transport_mode'].astype(str)
#routes['travel_time'] = routes['travel_time'].astype(str)
#routes['wait_time'] = routes['wait_time'].astype(str)
#snman.io.export_gdf(routes, os.path.join(outputs_path, 'routes.gpkg'))
routes

,from_id,to_id,travel_time
0,68252512,68172516,NaN
1,68252512,68172517,NaN
2,68252512,68182515,NaN
3,68252512,68182516,NaN
4,68252512,68182517,NaN
...,...,...,...
192,68252512,68342512,NaN
193,68252512,68342513,NaN
194,68252512,68342514,NaN
195,68252512,68342515,NaN


In [33]:
routes = tt_computers[MODE_CYCLING].compute_travel_details()
(routes['distance'][0] / 1000) / (routes['travel_time'][0].total_seconds() / 3600)

AttributeError: 'TravelTimeMatrixComputer' object has no attribute 'compute_travel_details'